# Cloud-native coastal waterline mapping

NOTE: Since the Planetary Computer Hub was retired this workflow is broken. Hopefully it will be fixed soon. 

In [ ]:
import sys

branch = "dev"
sys.path.insert(0, "../src")
from odc.stac import configure_rio

from coastmonitor.io.drive_config import configure_instance

is_local_instance = configure_instance(branch=branch)
configure_rio(cloud_defaults=True)

import logging
import os
import time
import warnings

import dask

dask.config.set({"dataframe.query-planning": False})
import dask.array as da
import dask.dataframe as dd
import dask_geopandas
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray  # noqa  # noqa  # noqa
import numpy as np
import pandas as pd
import xarray as xr
from astropy.convolution import convolve
from dask import delayed
from dask.distributed import performance_report
from dotenv import load_dotenv
from geopandas.array import GeometryDtype
from scipy import ndimage
from skimage import filters, morphology

from coastmonitor.dask_utils import generate_geometry_mask
from coastmonitor.geo.quadtiles import make_mercantiles
from coastmonitor.io.cloud import list_storage_location, write_block, write_table
from coastmonitor.io.eo import load_sentinel2_data
from coastmonitor.io.load import (
    infer_region_of_interest,
    retrieve_coastsat_classifier,
    retrieve_rois,
    retrieve_s2_tiles,
)
from coastmonitor.io.utils import name_block, name_table
from coastmonitor.transform.transform import (
    add_indices_preserve_nodata,
    mask_by_classes,
    mask_invalid_values,
)
from coastmonitor.xarray_utils import extract_and_set_nodata

In [ ]:
### Input params ###
DATE_RANGE = "2015-06-23/2024-11-01"
ROI = "NARRABEEN"  # NOTE: or bbox, like: (4.169655, 52.047265, 4.209394, 52.068902)
CLOUD_COVER = {"lt": 10}
INDICES = ["NDWI", "MNDWI", "NDVI", "NDMI", "BR"]
BANDS = ["blue", "green", "red", "nir", "swir16", "SCL"]
OVERWRITE = True

PROFILE_OPTIONS = {
    "driver": "COG",
    "dtype": "uint8",
    "compress": "LZW",
}
MERCANTILES_ZOOM_LEVEL = 10

SCL_CLASSES = {
    0: "No Data",
    1: "Saturated / Defective",
    2: "Dark Area Pixels",
    3: "Cloud Shadows",
    4: "Vegetation",
    5: "Bare Soils",
    6: "water",
    7: "Clouds low probability / Unclassified",
    8: "Clouds medium probability",
    9: "Clouds high probability",
    10: "Cirrus",
    11: "Snow / Ice",
}

SCL_CLASSES_TO_MASK = [
    "No Data",
    "Dark Area Pixels",
    "Clouds high probability",
    "Cirrus",
    # "Snow / Ice",  # do not mask because whitewater is often classified as ice
]

start_date_range, end_date_range = DATE_RANGE.split("/")
wop_storage_prefix = f"az://wop/{start_date_range}_to_{end_date_range}"
shoreline_storage_prefix = f"az://shorelines/{start_date_range}_to_{end_date_range}"

### NOTE: env vars
load_dotenv(override=True)

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_options = {"account_name": storage_account_name, "sas_token": sas_token}
gh_coastmonitor_token = os.getenv("GH_COASTMONITOR_TOKEN")

# Useful for fetching data on Planetary Computer
# NOTE: this should now be convered by odc.stac configure rio
# os.environ["GDAL_HTTP_MAX_RETRY"] = "3"

# Adjust logging level for azure and rasterio
logging.getLogger("azure").setLevel(logging.WARNING)
logging.getLogger("rasterio").setLevel(logging.WARNING)

In [ ]:
def clean_noise(
    image: np.ndarray, structure_size: int = 6, min_object_size: int = 100
) -> np.ndarray:
    """
    Apply morphological operations to clean noise from a binary image.

    Args:
        image (np.ndarray): Input binary array (image).
        structure_size (int, optional): Size of the square structuring element for morphological operations. Defaults to 6.
        min_object_size (int, optional): Minimum size of objects to retain in the image. Defaults to 100.

    Returns:
        np.ndarray: The cleaned image.

    Example:
        >>> image = np.random.randint(0, 2, (100, 100), dtype=bool)
        >>> cleaned_image = clean_noise_from_image(image, structure_size=5, min_object_size=50)
    """
    structure = morphology.square(structure_size)
    binary_opening = ndimage.binary_opening(image, structure=structure)
    image = morphology.remove_small_objects(
        binary_opening, min_size=min_object_size, connectivity=1
    )
    return image


def standard_deviation(
    image: np.ndarray, radius: int, nodata: float | int = np.nan
) -> np.ndarray:
    """
    Calculates the standard deviation of an image using a moving window of
    specified radius with astropy's convolution library.

    Args:
        image (np.ndarray): 2D array containing the pixel intensities of a single-band image.
        radius (int): Radius defining the moving window used to calculate the standard deviation.
                    For example, radius = 1 will produce a 3x3 moving window.
        nodata (float, optional): Value to replace NaN results with. Defaults to np.nan.

    Returns:
        np.ndarray: 2D array containing the standard deviation of the image.

    Example:
        >>> img = np.random.random((100, 100))
        >>> std_img = standard_deviation(img, radius=1)
    """

    # Create kernel once
    win_rows, win_cols = radius * 2 + 1, radius * 2 + 1
    kernel = np.ones((win_rows, win_cols))

    # Pre-calculate square of image
    image_sq = image**2

    # First pad the image and its square
    image_padded = np.pad(image, radius, "reflect")
    image_sq_padded = np.pad(image_sq, radius, "reflect")

    # Calculate std with uniform filters
    win_mean = convolve(
        image_padded,
        kernel,
        boundary="extend",
        normalize_kernel=True,
        nan_treatment="interpolate",
        preserve_nan=True,
    )
    win_sqr_mean = convolve(
        image_sq_padded,
        kernel,
        boundary="extend",
        normalize_kernel=True,
        nan_treatment="interpolate",
        preserve_nan=True,
    )
    win_var = win_sqr_mean - win_mean**2

    # Ignore RuntimeWarnings in the sqrt calculation
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="invalid value encountered in sqrt")
        win_std = np.sqrt(win_var)

    # Remove padding
    win_std = win_std[radius:-radius, radius:-radius]

    # After computing standard deviation, replace NaN values with nodata
    win_std[np.isnan(win_std)] = nodata

    return win_std


def apply_standard_deviation(
    da, input_core_dims, output_core_dims, radius=1, nodata=np.nan
):
    """
    Apply the standard deviation calculation to an xarray DataArray.

    Args:
        data_array (xr.DataArray): Input data.
        radius (int): The radius for the moving window.
        nodata (float): Value to replace NaN results with.
        dim (str or list): Dimensions over which to apply the ufunc.

    Returns:
        xr.DataArray: Result of the standard deviation calculation.
    """
    return xr.apply_ufunc(
        standard_deviation,
        da,
        input_core_dims=input_core_dims,
        output_core_dims=output_core_dims,
        vectorize=True,
        dask="parallelized",
        kwargs={"radius": radius, "nodata": nodata},
        output_dtypes=["f4"],
    )


def add_stdev_preserve_nodata(
    ds: xr.Dataset,
    exclude_vars: list | None = None,
    nodata: float | int = np.nan,
    radius: int = 1,
) -> xr.Dataset:
    """
    Calculate the standard deviation for all variables in an xarray dataset, excluding specified variables.

    Args:
        ds (xr.Dataset): Input dataset.
        exclude_vars (list, optional): List of variable names to exclude from the standard deviation calculation.
                                    Defaults to None.
        nodata (float, optional): Value to replace NaN results with. If not provided, it will be inferred.
                                        Defaults to None.
        radius (int, optional): Radius to compute the stdev. 1 equals an kernel of 3x3.
                                        Defaults to 1.

    Returns:
        xr.Dataset: Dataset with original variables and new variables containing standard deviations.

    Example:
        >>> ds = xr.Dataset({
        ...    "a": (["x", "y"], np.random.rand(4, 3)),
        ...    "b": (["x", "y"], np.random.rand(4, 3))
        ... })
        >>> result_ds = calculate_stdev_with_nodata(ds)
    """

    if exclude_vars is None:
        exclude_vars = []

    ds_for_stdev = ds.drop_vars(exclude_vars)
    rename_dict = {var: var + "_std" for var in list(ds_for_stdev.data_vars)}

    # If nodata_valu# If nodata_value is not explicitly provided, infer it
    if nodata is None:
        _, nodata = extract_and_set_nodata(ds, list(rename_dict.keys()), [])

    ds_stdev = apply_standard_deviation(
        ds_for_stdev,
        input_core_dims=[["y", "x"]],
        output_core_dims=[["y", "x"]],
        radius=radius,
        nodata=nodata,
    )

    # Construct new variable names and rename them
    ds_stdev = ds_stdev.rename(rename_dict)

    # Merge original and new data
    ds = xr.merge([ds, ds_stdev])

    ds, _ = extract_and_set_nodata(
        ds, list(rename_dict.keys()), list(rename_dict.values())
    )

    return ds


def classify_image(arr, classifier):
    output_shape = arr.shape[:2]

    arr = arr.reshape(-1, arr.shape[-1])
    nan_mask = np.isnan(arr).any(axis=1)  # computes nans along features index

    # arr with nan values to store result
    result = np.zeros(
        arr.shape[0],
    )
    result[:] = np.nan

    # don't bother classifying (and avoid error's) when all values are nan
    if nan_mask.all():
        return result.reshape(output_shape)

    predictions = classifier.predict(arr[~nan_mask])
    result[~nan_mask] = predictions
    result = result.reshape(output_shape)

    # TODO: erosion/dilation
    ...
    return result


def compute_otsu_threshold(arr):
    """
    Compute Otsu's threshold for a flattened array.

    Args:
        arr (np.ndarray): 1D array.

    Returns:
        float: The computed Otsu's threshold value.
    """
    nan_mask = np.isnan(arr)
    if nan_mask.all():
        return np.nan
    return filters.threshold_otsu(arr[~nan_mask])


def load_s2_shoreline_hypercube(
    bbox: gpd.GeoDataFrame(),
    query: dict,
    date_range: str,
    bands,
    indices,
    scl_classes_to_mask,
    buffer_roi_scattered,
    classifier,
):
    import rioxarray  # noqa

    SCL_CLASSES = {
        0: "No Data",
        1: "Saturated / Defective",
        2: "Dark Area Pixels",
        3: "Cloud Shadows",
        4: "Vegetation",
        5: "Bare Soils",
        6: "water",
        7: "Clouds low probability / Unclassified",
        8: "Clouds medium probability",
        9: "Clouds high probability",
        10: "Cirrus",
        11: "Snow / Ice",
    }

    if "SCL" not in bands:
        bands += "SCL"

    xx = load_sentinel2_data(bbox, date_range, query, bands)

    mask = mask_invalid_values(xx)
    scl_mask = mask_by_classes(xx.SCL, scl_classes_to_mask, SCL_CLASSES)

    geom_mask = generate_geometry_mask(xx.red, buffer_roi_scattered)

    mask = mask.merge(scl_mask)
    mask = mask.merge(geom_mask)

    def apply_clean_noise(da, input_core_dims, output_core_dims, output_dtypes):
        return xr.apply_ufunc(
            clean_noise,
            da,
            input_core_dims=input_core_dims,
            output_core_dims=output_core_dims,
            vectorize=True,
            dask="parallelized",
            output_dtypes=output_dtypes,
        )

    mask["SCL_mask"] = apply_clean_noise(
        mask.SCL_mask,
        input_core_dims=[["y", "x"]],
        output_core_dims=[["y", "x"]],
        output_dtypes=mask.SCL_mask.dtype,
    )

    xx = add_indices_preserve_nodata(
        xx,
        bands=bands,
        indices=indices,
        bands_to_rename={"swir16": "swir1"},
    )

    # invert NDWI, MNDWI, NDMI to match with CoastSat's classifier
    for var in ["NDWI", "MNDWI", "NDMI"]:
        xx[var] = xx[var] * -1

    scl = xx.SCL

    xx = xx.drop_vars("SCL")
    xx = add_stdev_preserve_nodata(xx, exclude_vars=[], radius=1, nodata=np.nan)

    mask_ = mask.drop_vars(["SCL_mask"]).to_array("variables").any("variables")

    # create a feature array
    feature_da = (
        xx.where(~mask_)
        .to_array("features")
        .chunk({"features": 20})
        .transpose("time", "y", "x", "features")
    )

    def apply_classify_image(da, input_core_dims, output_core_dims):
        return xr.apply_ufunc(
            classify_image,
            da,
            input_core_dims=input_core_dims,
            output_core_dims=output_core_dims,
            vectorize=True,
            dask="parallelized",
            output_dtypes="f4",
            kwargs={"classifier": classifier},
        )

    xx["CLASS"] = apply_classify_image(
        feature_da,
        input_core_dims=[["y", "x", "features"]],
        output_core_dims=[["y", "x"]],
    ).rename("CLASS")

    mask = mask.merge(~xx["CLASS"].isin([1, 3]).rename("CLASS_mask"))

    # # keep everything together
    xx = xx.merge(mask)

    # # computing otsu threshold masking out all potential noise sources
    otsu_threshold_mask = (
        mask.to_array("variable").any("variable").rename("otsu_threshold_mask")
    )

    shoreline_mask = (
        mask.drop_vars(["CLASS_mask", "SCL_mask"])
        .to_array("variable")
        .any("variable")
        .rename("shoreline_mask")
    )

    xx = xx.merge(otsu_threshold_mask)
    xx = xx.merge(shoreline_mask)

    def apply_otsu_threshold(da, input_core_dims):
        """
        Compute the global Otsu's threshold for an xarray DataArray along a specified dimension.

        Args:
            data_array (xr.DataArray): Input xarray DataArray.
            dim (str): Dimension along which to compute the Otsu's threshold.

        Returns:
            xr.DataArray: DataArray containing the global Otsu's threshold.
        """

        return xr.apply_ufunc(
            compute_otsu_threshold,
            da,
            input_core_dims=input_core_dims,
            vectorize=True,
            dask="parallelized",
            output_dtypes="f4",
        )

    t_otsu = apply_otsu_threshold(
        xx.MNDWI.where(~otsu_threshold_mask).chunk({"time": 1}),
        input_core_dims=[["y", "x"]],
    ).rename("otsu")

    # Compute the percentage of water pixels in scl and coassat to filter the images
    xx = xr.merge([xx, scl])

    coastsat_water_occurrence = (
        (xx.CLASS == 3).sum(dim=["y", "x"]) / xx.CLASS.count(dim=["y", "x"]) * 100
    )
    valid_data_mask = (xx.SCL != 0) & ~xx.SCL.isnull()
    scl_water_occurrence = (
        ((xx.SCL == 6) & valid_data_mask).sum(dim=["y", "x"])
        / valid_data_mask.sum(dim=["y", "x"])
        * 100
    )
    xx = xx.assign_coords(
        coastsat_water_occurrence=("time", coastsat_water_occurrence.data)
    )
    xx = xx.assign_coords(scl_water_occurrence=("time", scl_water_occurrence.data))
    xx = xx.assign_coords(t_otsu=("time", t_otsu.data))

    # NOTE: keep as ref because maybe change to otsu quality checking instead
    # mask_imgs_by_otsu_threshold = (xx.coords["t_otsu"] > -0.5) & (xx.coords["t_otsu"] < 0.5)
    mask_imgs_by_class = (
        (xx.coords["coastsat_water_occurrence"] > 95)
        | (xx.coords["scl_water_occurrence"] > 95)
    ).rename("img_mask")
    xx["final_mask"] = xx.shoreline_mask | mask_imgs_by_class

    xx["otsu"] = t_otsu > xx.where(~xx.final_mask).MNDWI
    return xx

In [ ]:
# NOTE: make a Dask Gateway or a LocalCluster depending on the instance type
if is_local_instance:
    from dask.distributed import Client

    logging.info("Launching local client...")
    client = Client(
        threads_per_worker=1,
        processes=True,
        local_directory="/tmp",
    )

    def silence_warnings():
        import warnings

        warnings.simplefilter("ignore", category=RuntimeWarning)

    client.run(silence_warnings)

else:
    import dask_gateway
    from distributed import PipInstall

    logging.info("Launching dask gateway client...")
    # NOTE: leave these params as they can be used if more memory is required
    # gateway = dask_gateway.Gateway()
    # cluster_options = gateway.cluster_options()
    # cluster_options["worker_memory"] = 16
    # cluster = gateway.new_cluster(cluster_options)

    cluster = dask_gateway.GatewayCluster()
    client = cluster.get_client()
    cluster.adapt(minimum=2, maximum=50)
    plugin = PipInstall(
        [
            f"git+https://{gh_coastmonitor_token}@github.com/floriscalkoen/coastmonitor.git@{branch}"
        ]
    )
    client.register_plugin(plugin)
    logging.info(f"Dashboard can be accessed at: {client.dashboard_link}.")
client

In [ ]:
# NOTE: LOAD DATA
s2_tiles = retrieve_s2_tiles().to_crs(4326)
rois = retrieve_rois().to_crs(4326)
# TODO: write STAC catalog for the coastal buffer
buffer = dask_geopandas.read_parquet(
    "az://coastline-buffer/osm-coastlines-buffer-2000m.parquet",
    storage_options=storage_options,
).compute()
quadtiles = make_mercantiles(zoom_level=MERCANTILES_ZOOM_LEVEL).to_crs(4326)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    classifier = retrieve_coastsat_classifier()

region_of_interest = infer_region_of_interest(ROI)

# NOTE: make an overlay with a coastline buffer to avoid querying data we do not need
buffer_aoi = gpd.overlay(buffer, region_of_interest[["geometry"]].to_crs(buffer.crs))

# TODO: add heuristic to decide which s2 tiles to use
s2_tilenames_to_process = gpd.sjoin(
    s2_tiles, buffer_aoi.to_crs(s2_tiles.crs)
).Name.unique()

In [ ]:
s2_tilenames_to_process

In [ ]:
# NOTE: process the shorelines per Sentinel 2 tile
dask_report_fp = f"{ROI}_dask-report.html"
with performance_report(dask_report_fp):
    start_time = time.time()
    for s2_tilename in s2_tilenames_to_process:
        s2_tile = (
            s2_tiles.loc[s2_tiles.Name == s2_tilename][["geometry"]]
            .explode(index_parts=False)
            .iloc[[0]]
        )
        bbox = tuple(s2_tile.total_bounds)
        query = {"eo:cloud_cover": CLOUD_COVER, "s2:mgrs_tile": {"eq": s2_tilename}}

        # NOTE: the result will be reprojected to this raster. # TODO: consider if it
        # would be better to do a trim_outer_nan's here.
        # BUG: raster cannot be made with one band, so we add green.
        template_raster = load_sentinel2_data(
            bbox, "2023-06-23/2023-11-01", query, ["red"]
        )
        template_raster = template_raster.red.isel(time=0)

        s2_tile_by_aoi = gpd.overlay(
            s2_tile, region_of_interest[["geometry"]].to_crs(s2_tile.crs)
        )

        # NOTE: the S2 tiles are too large to process as a whole, so here smaller tiles
        # quadkeys are created to process in smaller chunks
        tiles = gpd.sjoin(quadtiles, s2_tile_by_aoi).drop(columns=["index_right"])
        tiles = (
            gpd.sjoin(tiles, buffer.to_crs(4326))
            .drop(columns=["index_right", "EPSG"])
            .drop_duplicates("quadkey")
        )
        # TODO: replace prints with tqdm
        logging.info(
            f"Start processing next S2 tile: {s2_tilename} that spans"
            f" {len(tiles)} quadtiles."
        )

        # NOTE: process per quadkey
        for _, tile in tiles.iloc[[0]].iterrows():
            # NOTE: don't process data that is already processed.
            if not OVERWRITE:
                list_files_prefix = (
                    f"{start_date_range}_to_{end_date_range}*s2={s2_tilename}*"
                )
                files = list_storage_location(
                    f"az://wop/{start_date_range}_to_{end_date_range}/",
                    storage_options=storage_options,
                    prefix=list_files_prefix,
                )
                is_processed = any(
                    tile.quadkey in f and s2_tilename in f for f in files
                )
                if is_processed:
                    continue

            logging.info(f"Start processing next quadtile: {tile.quadkey}")

            roi = tile.to_frame().transpose().set_geometry("geometry", crs=tiles.crs)
            roi = gpd.overlay(roi, region_of_interest[["geometry"]].to_crs(roi.crs))
            roi = rois.loc[["NARRABEEN"]]

            # NOTE: distribute the buffer for the region of interest to the workers
            buffer_roi = gpd.overlay(buffer, roi.to_crs(buffer.crs)).dissolve()
            # NOTE: in rare cases there is no overlap beteween the tile and the buffer (at boundaries ROI)
            if buffer_roi.empty:
                continue
            buffer_roi_scattered = client.scatter(buffer_roi, broadcast=True)

            # NOTE: query datacube from STAC
            bbox = tuple(roi.total_bounds)

            xx = load_s2_shoreline_hypercube(
                bbox=bbox,
                query=query,
                date_range=DATE_RANGE,
                bands=BANDS,
                indices=INDICES,
                scl_classes_to_mask=SCL_CLASSES_TO_MASK,
                buffer_roi_scattered=buffer_roi_scattered,
                classifier=classifier,
            )

            xx = xx[["otsu", "MNDWI", "final_mask", "shoreline_mask"]]

            def map_shorelines(da):
                # TODO: check how to make rioxarray available to workers by default
                import rioxarray  # noqa

                from coastmonitor.dea_tools import subpixel_contours

                # NOTE: ruff wants to change '== false' to 'is False', which breaks the condition
                ds = da.to_dataset("band")
                df = subpixel_contours(
                    ds.MNDWI.where(ds.final_mask == False).to_numpy(),  # noqa: E712
                    z_values=ds.coords["t_otsu"].to_numpy(),
                    crs=ds.rio.crs.to_epsg(),
                    affine=ds.rio.transform(),
                )
                df["time"] = ds.coords["time"].item()
                return df

            META = gpd.GeoDataFrame(
                {
                    "z_value": pd.Series(dtype=str),
                    "geometry": pd.Series(dtype=GeometryDtype),
                    "time": pd.Series(dtype="i8"),
                }
            )

            dfs = []
            for da in (
                xx[["MNDWI", "final_mask"]]
                .to_array("band")
                .transpose("time", "band", "y", "x")
            ):
                df = delayed(map_shorelines)(da)
                dfs.append(df)
            shorelines = dd.from_delayed(dfs, meta=META)

            # TODO: discuss if we need to save multiple time ranges
            wop = (xx.otsu.where(~xx.final_mask)).mean("time").rename("wop")
            nodata_value = 255
            wop = wop * 100
            wop = wop.where(~np.isnan(wop), nodata_value)

            wop = (
                wop.astype(np.uint8)
                .rio.write_nodata(nodata_value)
                .rio.set_spatial_dims(x_dim="x", y_dim="y")
            )
            wop.attrs = xx.attrs

            wop, shorelines = dask.compute(*[wop, shorelines])
            wop = wop.rio.reproject_match(template_raster, nodata=nodata_value)
            # NOTE: the outer nan's can be trimmed, but to create tiles with consistent
            # shape they are currenlty included when writing the results to the cloud container.
            # trimmed_wop = trim_outer_nans(wop,nodata=nodata_value).astype(np.uint8)
            wop_href = name_block(
                wop,
                storage_prefix=wop_storage_prefix,
                name_prefix=f"s2={s2_tilename}_qk={tile.quadkey}",
            )

            write_block(
                wop,
                wop_href,
                storage_options=storage_options,
                profile_options=PROFILE_OPTIONS,
            )

            shorelines = shorelines.to_crs(4326)
            shorelines_href = name_table(
                shorelines,
                storage_prefix=shoreline_storage_prefix,
                name_prefix=f"s2={s2_tilename}_qk={tile.quadkey}",
            )
            write_table(shorelines, shorelines_href, storage_options=storage_options)

            logging.info("Done!")
            elapsed_time = time.time() - start_time
            logging.info(
                "Time (H:M:S):"
                f" {time.strftime('%H:%M:%S', time.gmtime(elapsed_time))}"
            )

In [ ]:
wop.rio.reproject(4326, nodata=255).where(lambda xx: xx != 255).hvplot(
    x="x", y="y", rasterize=True, geo=True, tiles="EsriImagery", width=600
)

In [ ]:
shorelines = shorelines.assign(
    length=shorelines.to_crs(shorelines.estimate_utm_crs()).geometry.length
)
shorelines = shorelines.assign(
    time=pd.DatetimeIndex(shorelines.time).strftime("%Y-%m-%d")
).astype({"z_value": "f4"})

In [ ]:
shorelines_ = shorelines.loc[(shorelines.z_value > -0.3) & (shorelines.z_value < 0.3)]
shorelines_.sort_values("length", ascending=False).iloc[:50].to_crs(4326).explore(
    column="time"
)